In [1]:
'''


'''
def create_data(file_path: str):

    data = {}

    with open(file_path, 'r') as f:
        if f == None:
            print('File not found')
            return None
                
        data["num_subjects"] = int(f.readline())
        data["num_students_per_subject"] = [int(x) for x in f.readline().split()]
        data["num_rooms"] = int(f.readline())
        data["num_seats_per_room"] = [int(x) for x in f.readline().split()]
        data["num_pairs"] = int(f.readline())
        data["conflicts"] = []
        for i in range (data["num_pairs"]):
            s1, s2 = [int(x) for x in f.readline().split()]
            data["conflicts"].append((s1, s2))
        
    return data

In [12]:
from ortools.linear_solver import pywraplp
import math

status_dict = {
    pywraplp.Solver.OPTIMAL: "OPTIMAL",
    pywraplp.Solver.FEASIBLE: "FEASIBLE",
    pywraplp.Solver.INFEASIBLE: "INFEASIBLE",
    pywraplp.Solver.UNBOUNDED: "UNBOUNDED",
    pywraplp.Solver.ABNORMAL: "ABNORMAL",
    pywraplp.Solver.NOT_SOLVED: "NOT_SOLVED",
}

def solve_with_mip(num_subjects: int, num_rooms: int, nums_student_per_subject: int, 
                   num_seats_per_room: int, subject_pairs: int,
                   num_sections_per_day: int = None, num_days: int = None, 
                   save_model: bool = False, time_limit: int = 600):
    
    if (not num_sections_per_day):
        num_sections_per_day = 4

    if (not num_days):
        num_days = math.ceil(num_subjects / num_sections_per_day)
        
    max_sum_sections = num_subjects
    
    # Create a new linear solver
    solver = pywraplp.Solver('Scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}

    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                x[(subject, room, section_id)] = solver.IntVar(0, 1, 'x[%i, %i, %i]' % (subject, room, section_id))
                
    y = solver.IntVar(0, max_sum_sections, 'y')
    
    # Constraints 1: Each subject must be assigned once
    for subject in range(num_subjects):
        temp = []
        
        for room in range(num_rooms):
            c = solver.Sum([x[(subject, room, section_id)] for section_id in range(max_sum_sections)])
            temp.append(c)
        
        solver.Add(solver.Sum(temp) == 1)        
        
    # Constraints 2: Each room must have most one subject in each section
    for room in range(num_rooms):
        for section_id in range(max_sum_sections):
            solver.Add(solver.Sum([x[(subject, room, section_id)] 
                                   for subject in range(num_subjects)]) <= 1)

    # Constraints 3: Each subject assigned to a room must have enough seats
    for subject in range(num_subjects):
        for room in range(num_rooms):
            solver.Add(solver.Sum([x[(subject, room, section_id)] * nums_student_per_subject[subject] 
                                   for section_id in range(max_sum_sections)]) <= num_seats_per_room[room])
    
    # Constraints 4: Two subjects cannot be assigned to the same room in the same section
    for section_id in range(max_sum_sections):
        for subject1, subject2 in subject_pairs:
            solver.Add(solver.Sum([x[(subject1, room, section_id)] + x[(subject2, room, section_id)]
                                   for room in range(num_rooms)]) <= 1)

    # Constraints 5: Minimize the number of sections
    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                solver.Add(y >= x[(subject, room, section_id)] * section_id)
    
    # Set time limit
    solver.set_time_limit(time_limit * 1000)

    # Objective function
    solver.Minimize(y)
    
    if save_model:
        with open('mip.mps', 'w') as f:
            f.write(solver.ExportModelAsMpsFormat(False, False))
    
    # Solve the problem
    status = solver.Solve()
    
    print("Status = ", status_dict[status], end='\n')
    
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        solution = []
    
        print('Objective value =', solver.Objective().Value())
        for subject in range(num_subjects):
            for room in range(num_rooms):
                for section_id in range(max_sum_sections):
                    if x[(subject, room, section_id)].solution_value() > 0:
                        print('Subject %i is assigned to room %i in section %i' % (subject, room, section_id))
                        
                        solution.append((subject, room, section_id // num_sections_per_day, section_id % num_sections_per_day))
        
        solution.sort(key=lambda x: x[2] * num_sections_per_day + x[3])
        
        solution_str = ''        
        solution_str += "subjects,number_student,rooms,num_seat,day,section" + "\n"
        for subjects, room, day, section in solution:
            solution_str += str(subjects) + "," + str(nums_student_per_subject[subjects]) + "," + str(room) + "," + \
                str(num_seats_per_room[room]) + "," + str(day) + "," + str(section) + "\n"

        return solution_str
    elif status == pywraplp.Solver.NOT_SOLVED:
        print('Time Limit')
        return None
    
    else:
        print('The problem does not have an optimal solution.')
        

In [13]:
import os

path_to_data = '../data/set_conflicts/'

file_name = 'data_20_4_60_(0).txt'

data = create_data(path_to_data + file_name)

solution_str = solve_with_mip(num_subjects=data["num_subjects"], num_rooms=data["num_rooms"], 
                        nums_student_per_subject=data["num_students_per_subject"],
                        num_seats_per_room=data["num_seats_per_room"], subject_pairs=data["conflicts"],
                        time_limit=120)

output_path = "../solution/mip/" + file_name[:-4] + "_solution.csv"

with open(output_path, 'w') as f:
    f.write(solution_str)

Status =  OPTIMAL
Objective value = 5.0
Subject 0 is assigned to room 0 in section 1
Subject 1 is assigned to room 3 in section 2
Subject 2 is assigned to room 0 in section 4
Subject 3 is assigned to room 1 in section 4
Subject 4 is assigned to room 1 in section 5
Subject 5 is assigned to room 2 in section 2
Subject 6 is assigned to room 0 in section 3
Subject 7 is assigned to room 1 in section 1
Subject 8 is assigned to room 0 in section 2
Subject 9 is assigned to room 3 in section 1
Subject 10 is assigned to room 2 in section 4
Subject 11 is assigned to room 3 in section 3
Subject 12 is assigned to room 1 in section 0
Subject 13 is assigned to room 1 in section 3
Subject 14 is assigned to room 1 in section 2
Subject 15 is assigned to room 3 in section 5
Subject 16 is assigned to room 2 in section 1
Subject 17 is assigned to room 0 in section 5
Subject 18 is assigned to room 3 in section 4
Subject 19 is assigned to room 2 in section 3


In [4]:
import os

path_to_data = '../data/set_conflicts/'

file_name = 'data_20_4_60_(1).txt'

data = create_data(path_to_data + file_name)

solution_str = solve_with_mip(num_subjects=data["num_subjects"], num_rooms=data["num_rooms"], 
                        nums_student_per_subject=data["num_students_per_subject"],
                        num_seats_per_room=data["num_seats_per_room"], subject_pairs=data["conflicts"])

output_path = "../solution/mip/" + file_name[:-4] + "_solution.csv"

with open(output_path, 'w') as f:
    f.write(solution_str)

Objective value = 5.0
Subject 0 is assigned to room 1 in section 2
Subject 1 is assigned to room 0 in section 3
Subject 2 is assigned to room 0 in section 2
Subject 3 is assigned to room 2 in section 3
Subject 4 is assigned to room 3 in section 5
Subject 5 is assigned to room 3 in section 3
Subject 6 is assigned to room 1 in section 3
Subject 7 is assigned to room 3 in section 1
Subject 8 is assigned to room 1 in section 5
Subject 9 is assigned to room 3 in section 4
Subject 10 is assigned to room 1 in section 1
Subject 11 is assigned to room 0 in section 0
Subject 12 is assigned to room 1 in section 0
Subject 13 is assigned to room 2 in section 5
Subject 14 is assigned to room 3 in section 0
Subject 15 is assigned to room 1 in section 4
Subject 16 is assigned to room 2 in section 1
Subject 17 is assigned to room 3 in section 2
Subject 18 is assigned to room 0 in section 1
Subject 19 is assigned to room 0 in section 5
